# ED-IP Prediction with Random Forests

The goal of the model is to predict whether or not a member is going to have an ED or IP visit in the next 180 following the latest claim stratification. 

In [1]:
# Importing Necessary dependencies
import pandas as pd,datetime
import snowflake.connector as sf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time
import os
from pandas_profiling import ProfileReport

### Extracting Data Frame from Snowflake Tables


In [2]:
# Snowflake credentials stored in environment variables

username = os.getenv('Snowflake_User')
password = os.getenv('Snowflake_password')
account = os.getenv('Snowflake_account')

# Define warehouse, if neccessary
warehouse = 'DEVELOPER_BASIC'

# Define Database, if not defined in SQL request
#database = 'VESTA_STAGING'

# Create connection object for Snowflake connection
conn = sf.connect(user = username, password = password, account = account, warehouse = warehouse)

# Execution function
def execute_query(connection,query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close

try:
    # If defining a database, uncomment code set and add database in connection parameter
    #sql = 'use {}'.format(database)
    #execute_query(conn,sql)1011
    
    # Define warehouse to use in Snowflake
    sql = 'use warehouse {}'.format(warehouse)
    execute_query(conn,sql)
    
    print('Successful Connection')
    
    # Query to Snowflake
    sql = '''WITH EDIP AS ( //This is sub table for a self join

    SELECT 
        *
    FROM "VESTA_DEVELOPMENT"."CLAIMS_REPORTING"."CCA_MEM_PROFILE_IP_ER_SNF" //THIS NEEDS TO CHANGE BASED ON CLIENT
    WHERE MEASURE = 'ED' or MEASURE = 'IP' 

    ),

    EDIPTABLE AS ( //This table shows the Member ID, date start, and the number of ED/IP in the next 6 months

    SELECT
        SCORE.MEMBER_ID,
        TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) AS DATE_START,
        COUNT(DISTINCT EDIP.DOS_FROM) AS ED_IP_VISITS_IN_NEXT_6_MONTHS
    FROM "VESTA_DEVELOPMENT"."ANALYST_SANDBOX"."CLNT_STRAT_VIP" SCORE
        LEFT JOIN EDIP 
            ON SCORE.MEMBER_ID = EDIP.MEMBER_ID
                AND EDIP.DOS_FROM > TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01'))
                AND DATEDIFF(days,TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')),EDIP.DOS_FROM) >= 45
                AND DATEDIFF(days, TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')),EDIP.DOS_FROM) <= 180
    GROUP BY SCORE.MEMBER_ID,
        DATE_START 

    )

SELECT
    //TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) AS DATE_START,
    //DATA_DATE_START,
    //SCORE.MEMBER_ID,
    CASE WHEN EDIPTABLE.ED_IP_VISITS_IN_NEXT_6_MONTHS > 0 THEN 1 ELSE 0 END as ED_IP_VISIT,
    ACSC__COUNT,
    ACSC__SCORE,
    ACSC_A_FIB_AND_FLUTTER,
    ACSC_ALCOHOL_RELATED,
    ACSC_ANEMIA,
    ACSC_ANGINA,
    ACSC_ASTHMA,
    ACSC_CELLULITIS,
    ACSC_CONGESTIVE_HEART_FAILURE,
    ACSC_CONSTIPATION,
    ACSC_CONVULSION_EPILEPSY,
    ACSC_COPD,
    ACSC_DECUBITI_STAGE_3_,
    ACSC_DEHYDRATION_GASTROENTERITIS,
    ACSC_DIABETES_COMPLICATIONS,
    ACSC_DYSPEPSIA,
    ACSC_ENT_INFECTION,
    ACSC_HYPERTENSION,
    ACSC_HYPOGLYCEMIA,
    ACSC_HYPOKALEMIA,
    ACSC_INFLUENZA_PNEUMONIA,
    ACSC_MIGRAINE_HEADACHE,
    ACSC_NUTRITION_DEFICIENT,
    ACSC_PERFORATED_BLEEDING_ULCER,
    ACSC_PROXIMAL_FEMUR_FRACTURE,
    ACSC_PYELONEPHRITIS,
    ACSC_UTI,
    ACSC_VACCINE_PREVENTABLE_DX,
    DATEDIFF(year,DOB, TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01'))) as AGE,
    AMB_ACSC_COST,
    AMB_ACSC_COUNT,
    BH__COUNT,
    BH__SCORE,
    BH_ALTERED_MENTAL_STATE,
    BH_ALZHEIMERS_DEMENTIA,
    BH_ANXIETY,
    BH_BI_POLAR,
    BH_DEPRESSION,
    BH_SCHIZOPHRENIA,
    BH_SUBABUSE,
    CRN__COUNT,
    CRN_AFIB,
    CRN_ASTHMA,
    CRN_CARDIOVASCULAR_DX,
    CRN_CHRONIC_KIDNEY_DISEASE,
    CRN_CONGESTIVE_HEART_FAILURE,
    CRN_COPD,
    CRN_DIABETES_W__ACUTE_COMP,
    CRN_DIABETES_W__CHRONIC_COMP,
    CRN_DIABETES_W_OUT_COMP,
    CRN_FALLS,
    CRN_GASTRO_ESOPH_REFLUX,
    CRN_HIP_FRACTURE,
    CRN_HTN,
    CRN_OBESITY,
    CRN_OSTEOPOROSIS,
    CRN_PARKINSONS_DISEASE,
    CRN_PRESSURE_ULCER,
    CRN_PRIOR_MI,
    CRN_PRIOR_STROKE,
    CRN_SCORE,
    CRN_SLEEP_APNEA,
    CRN_SMOKING,
    CRN_UTI,
    DYAD_CKD_DD,
    DYAD_CKD_OP,
    DYAD_COPD_DD,
    DYAD_COPD_HF,
    DYAD_COPD_OP,
    DYAD_COUNT,
    DYAD_DM_CKD,
    DYAD_DM_OP,
    DYAD_HBP_HF,
    DYAD_HF_CKD,
    ED_ACSC_COST,
    ED_ACSC_COUNT,
    GENDER,
    CASE WHEN "GROUP" = 'E' THEN 'A' ELSE "GROUP" END as "GROUP",
    HMKR_ACSC_COST,
    HMKR_ACSC_COUNT,
    HTI_RISK_SCORE_V2_1,
    IP_ACSC_COST,
    IP_ACSC_COUNT,
    IP_READMIT_ACSC_COST,
    IP_READMIT_ACSC_COUNT,
    IP_RHB_ACSC_COST,
    IP_RHB_ACSC_COUNT,
    CASE WHEN LANGUAGE_SPOKEN is NULL THEN 'Unknown'
         WHEN LANGUAGE_SPOKEN = 'English' THEN 'English'
         WHEN LANGUAGE_SPOKEN = 'Chinese' THEN 'Chinese'
         WHEN LANGUAGE_SPOKEN = 'Spanish' THEN 'Spanish'
         WHEN LANGUAGE_SPOKEN = 'Russian' THEN 'Russian'
    ELSE 'Other' END AS LANGUAGE_SPOKEN_CLEAN,
    NI_COST_DENT,
    NI_COST_ED,
    NI_COST_HM,
    NI_COST_HMKR,
    NI_COST_HS,
    NI_COST_IP,
    NI_COST_IP_RHB,
    NI_COST_OP,
    NI_COST_OTH,
    NI_COST_PCA_T1020,
    NI_COST_PCA_T1019,
    NI_COST_PR,
    NI_COST_PSYC,
    NI_COST_RX,
    NI_COUNT_DENT,
    NI_COUNT_ED,
    NI_COUNT_HM,
    NI_COUNT_HMKR,
    NI_COUNT_HS,
    NI_COUNT_IP,
    NI_COUNT_IP_RHB,
    NI_COUNT_OP,
    NI_COUNT_OTH,
    NI_COUNT_PCA_T1020,
    NI_COUNT_PCA_T1019,
    NI_COUNT_PR,
    NI_COUNT_PSYC,
    NI_COUNT_RX,
    NON_IMPACTABLE_CLAIM_COUNT,
    OP_ACSC_COST,
    OP_ACSC_COUNT,
    CAST(PART_C_RISK_SCORE as FLOAT) as PART_C_RISK_SCORE,
    PCA_T1020_ACSC_COUNT,
    PCA_T1020_ACSC_COST,
    PCA_T1019_ACSC_COUNT,
    PCA_T1019_ACSC_COST,
    PR_ACSC_COST,
    PR_ACSC_COUNT,
    CASE WHEN RC is NULL THEN 'UNDEFINED' ELSE RC END AS RC_CLEAN,
    SNF_COST,
    SNF_COUNT,
    TOTAL_IMPACTABLE_COST,
    TOTAL_IMPACTABLE_COST_PRO,
    TOTAL_NON_IMPACTABLE_COST
FROM "VESTA_DEVELOPMENT"."ANALYST_SANDBOX"."CLNT_STRAT_VIP" SCORE
    LEFT JOIN EDIPTABLE
        ON SCORE.MEMBER_ID = EDIPTABLE.MEMBER_ID
            AND TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) = EDIPTABLE.DATE_START
WHERE SCORE.CLNT = 'CCA' //THIS NEEDS TO CHANGE BASED ON CLIENT
    AND TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) 
    < TO_DATE(CONCAT(LEFT(CURRENT_DATE-210,7),'-01')) //This is looking at files that have had a reasonable amount of time to process'''
    cursor = conn.cursor()
    cursor.execute(sql)
    
    # Dataframe creation
    df = pd.DataFrame.from_records(iter(cursor), columns = [x[0] for x in cursor.description])
    
    print('Successful DataFrame Created')
    
    cursor.close
    
except Exception as e:
        print(e)
        
finally:
    conn.close
    
print('Ready for Cleaning')

Successful Connection
Successful DataFrame Created
Ready for Cleaning


### Cleaning Operations

In [3]:
# Find features with missing values
sumdf = pd.DataFrame(df.isna().sum())

features_to_drop = []
for row in sumdf.iterrows():
    if row[1][0] != 0:
        features_to_drop.append(row[0])

# Review features with missing values
print('These are the features that initially had missing values within the data frame: \n',features_to_drop,'\n\n')

Nulls_to_correct = ['ACSC_A_FIB_AND_FLUTTER', 'ACSC_ALCOHOL_RELATED', 'ACSC_ANEMIA', 'ACSC_ANGINA', 
                    'ACSC_ASTHMA', 'ACSC_CELLULITIS', 'ACSC_CONSTIPATION', 'ACSC_CONVULSION_EPILEPSY', 
                    'ACSC_DECUBITI_STAGE_3_', 'ACSC_DEHYDRATION_GASTROENTERITIS', 'ACSC_ENT_INFECTION', 
                    'ACSC_HYPOGLYCEMIA', 'ACSC_HYPOKALEMIA', 'ACSC_MIGRAINE_HEADACHE', 'ACSC_PERFORATED_BLEEDING_ULCER', 
                    'ACSC_PROXIMAL_FEMUR_FRACTURE', 'ACSC_PYELONEPHRITIS', 'ACSC_VACCINE_PREVENTABLE_DX', 'BH_SUBABUSE', 
                    'CRN_PRIOR_MI', 'CRN_SMOKING', 'NI_COST_DENT', 'NI_COST_HMKR', 'NI_COST_HS', 'NI_COST_IP_RHB', 
                    'NI_COST_PSYC', 'NI_COUNT_DENT', 'NI_COUNT_HMKR', 'NI_COUNT_HS', 'NI_COUNT_IP_RHB', 'NI_COUNT_PSYC'] 

# Fill selected features with 0 value
for col in Nulls_to_correct:
    df[col] = df[col].fillna(0)
    
# Check for missing values and drop columns with missing values
sumdf = pd.DataFrame(df.isna().sum())

features_to_drop = []
for row in sumdf.iterrows():
    if row[1][0] != 0:
        features_to_drop.append(row[0])

print('These columns were dropped after because missing values were not corrected :\n', features_to_drop,'\n\n')        
df = df.drop(columns = features_to_drop)

# Convert object datatypes to dummy variables
object_list = []

for col in df.columns:
    if df[col].dtypes == 'object':
        object_list.append(col)
        
        
print('These are the features that were converted to dummy variables: \n',object_list,'\n\n')
df = pd.get_dummies(df, columns = object_list, drop_first = True)

print(df.dtypes)

original_memory = df.memory_usage().sum()
print(f'Memory Usage of Dataframe: {df.memory_usage().sum()} bytes')

df.head()

These are the features that initially had missing values within the data frame: 
 ['ACSC_A_FIB_AND_FLUTTER', 'ACSC_ALCOHOL_RELATED', 'ACSC_ANEMIA', 'ACSC_ANGINA', 'ACSC_ASTHMA', 'ACSC_CELLULITIS', 'ACSC_CONSTIPATION', 'ACSC_CONVULSION_EPILEPSY', 'ACSC_DECUBITI_STAGE_3_', 'ACSC_DEHYDRATION_GASTROENTERITIS', 'ACSC_ENT_INFECTION', 'ACSC_HYPOGLYCEMIA', 'ACSC_HYPOKALEMIA', 'ACSC_MIGRAINE_HEADACHE', 'ACSC_PERFORATED_BLEEDING_ULCER', 'ACSC_PROXIMAL_FEMUR_FRACTURE', 'ACSC_PYELONEPHRITIS', 'ACSC_VACCINE_PREVENTABLE_DX', 'CRN_PRIOR_MI', 'CRN_SMOKING', 'NI_COST_HS', 'NI_COST_IP_RHB', 'NI_COST_PSYC', 'NI_COUNT_HS', 'NI_COUNT_IP_RHB', 'NI_COUNT_PSYC'] 


These columns were dropped after because missing values were not corrected :
 [] 


These are the features that were converted to dummy variables: 
 ['GENDER', 'GROUP', 'LANGUAGE_SPOKEN_CLEAN', 'RC_CLEAN'] 


ED_IP_VISIT                        int64
ACSC__COUNT                      float64
ACSC__SCORE                      float64
ACSC_A_FIB_AND_FLU

,ED_IP_VISIT,ACSC__COUNT,ACSC__SCORE,ACSC_A_FIB_AND_FLUTTER,ACSC_ALCOHOL_RELATED,ACSC_ANEMIA,ACSC_ANGINA,ACSC_ASTHMA,ACSC_CELLULITIS,ACSC_CONGESTIVE_HEART_FAILURE,...,GROUP_B,GROUP_C,GROUP_D,LANGUAGE_SPOKEN_CLEAN_Other,LANGUAGE_SPOKEN_CLEAN_Russian,LANGUAGE_SPOKEN_CLEAN_Spanish,LANGUAGE_SPOKEN_CLEAN_Unknown,RC_CLEAN_Institutional,RC_CLEAN_NHC,RC_CLEAN_Other
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,0,0,0,0,1,0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,1,0,0,0,0,1,0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,1,0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,1,0,0,1,0


In [4]:
#convert data types changing all float64 to float32
df[df.select_dtypes(np.float64).columns] = df.select_dtypes(np.float64).astype(np.float32)
new_memory = df.memory_usage().sum()
print(f'Memory Usage of Original Dataframe: {original_memory} bytes')
print(f'Memory Usage of New Dataframe: {new_memory} bytes')
print(f'Memory usage reduced by:{round((original_memory-new_memory)/original_memory * 100,0)}%')

Memory Usage of Original Dataframe: 84279116 bytes
Memory Usage of New Dataframe: 43269800 bytes
Memory usage reduced by:49.0%


### Option to remove low variability features

This block is here to run as optional pre-processing. 

In [5]:
# Drop low variability columns
df_var = df.var()
df.columns.to_list()

features_to_drop = []

for i in range(len(df.columns.to_list())):
    #print(df.columns.to_list()[i],df_var[i])
    if df_var[i] == 0 and df.columns.to_list()[i] != 'ED_IP_VISIT':
        features_to_drop.append(df.columns.to_list()[i])

        
print('These are the features that were dropped because of low variability: \n',features_to_drop,'\n\n')
        
df = df.drop(columns = features_to_drop)
print(f'Memory Usage of Dataframe: {df.memory_usage().sum()} bytes')

These are the features that were dropped because of low variability: 
 [] 


Memory Usage of Dataframe: 43269800 bytes


### Spliting, Scaling, and SMOTE (Synthetic Minority Oversampling Technique)

In [ ]:
# Split the data set
X = df[[col for col in df.columns if col != 'ED_IP_VISIT']] #independent variables
y = df[[col for col in df.columns if col == 'ED_IP_VISIT']] #dependent variable
y = y.values.flatten()

# Define MinMax Scaler
scaler = MinMaxScaler()

# Transform data
X = scaler.fit_transform(X)

# Split X and y into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state = 2)

# Smote for balancing the training data set
smote = SMOTE(random_state = 2)
X_train,y_train = smote.fit_resample(X_train, y_train)
print(f'Memory Usage of Dataframe: {df.memory_usage().sum()} bytes')

### Creating and Training Random Forest Classifier

In [ ]:
%%time
# Create a Random Forest Classifier
clf=RandomForestClassifier(n_estimators = 2000,min_samples_split = 2, min_samples_leaf = 1,
                           max_depth = 50, bootstrap = False, n_jobs = -1,random_state = 2)

# Train the model using the training sets
clf.fit(X_train,y_train)
print(f'Memory Usage of Dataframe: {df.memory_usage().sum()} bytes')

### Reporting on Performace

In [ ]:
# Create probabilities from the model on test data
y_prob = clf.predict_proba(X_test)[:,1]

# Store probabilites in Datafram for threshold analysis
threshold_list = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,.7,.75,.8,.85,.9,.95,.99]
accuracy_list = []
precision_list = []
recall_list = []
for threshold in threshold_list:
    y_pred = [1 if result >= threshold else 0 for result in y_prob]
   
    #Calulating Metrics
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    
    #print('Thereshold: ',threshold)
    #print("Accuracy: ",accuracy)
    #print("Precision: ",precision)
    #print("Recall: ",recall)
    
metric_df = pd.DataFrame()
metric_df['Threshold'] = threshold_list
metric_df['Accuracy'] = accuracy_list
metric_df['Precision'] = precision_list
metric_df['Recall'] = recall_list
metric_df['F1'] = (2 * metric_df['Precision'] * metric_df['Recall']) / (metric_df['Precision'] + metric_df['Recall'])
metric_df['Acc + Recall'] = metric_df['Accuracy'] + metric_df['Recall']

metric_df


In [ ]:
metric_df

### Confusion Matrix for Threshold with Max Accuracy + Recall

In [ ]:
#Find the max accuracy and recall from the Metric Table and corresponding Threshold
threshold = metric_df[metric_df['Acc + Recall'] == metric_df['Acc + Recall'].max()]['Threshold'].item()

y_pred = [1 if result >= threshold else 0 for result in y_prob]


# Creating confusion maxtrix
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

%matplotlib inline
class_names=[0,1]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# Create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")

# Axis labels
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# Show the plot
plt.show()


#Calulating Metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
    
print('Thereshold: ',threshold)
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)
print('F1: ', (2*precision*recall)/(precision+recall))

### ROC and AUC and Precision-Recall Curve

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test,  y_prob)

# Print(thresholds)
auc = metrics.roc_auc_score(y_test, y_prob)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.plot([0, 1], [0, 1], color="red", linestyle="--")

# Axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Show the legend
plt.legend(loc=4)

# Show the plot
plt.show()

# Calculate precision and recall for each threshold
precision, recall, _ = metrics.precision_recall_curve(y_test, y_prob)

# Calculate scores
f1, auc = metrics.f1_score(y_test, y_pred), metrics.auc(recall, precision)

# Plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(recall, precision, marker='.', label='Random Forest')

# Axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')

# Show the legend
plt.legend()

# Show the plot
plt.show()

### Ensemble of Random Forests

In [6]:
forest_dict = {}

for i in range(0,10):
    start_time = time.time()
    # Split the data set
    X = df[[col for col in df.columns if col != 'ED_IP_VISIT']] #independent variables
    y = df[[col for col in df.columns if col == 'ED_IP_VISIT']] #dependent variable
    y = y.values.flatten()

    # Define MinMax Scaler
    scaler = MinMaxScaler()

    # Transform data
    X = scaler.fit_transform(X)

    # Split X and y into training and testing sets
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state = i)

    # Smote for balancing the training data set
    smote = SMOTE(random_state = i)
    X_train,y_train = smote.fit_resample(X_train, y_train)
    
    # Create a Random Forest Classifier
    clf=RandomForestClassifier(n_estimators = 2000,min_samples_split = 2, min_samples_leaf = 1,
                               max_depth = 50, bootstrap = False, n_jobs = -1,random_state = i)

    # Train the model using the training sets
    clf.fit(X_train,y_train)
    
    #Store Random Forest
    forest_dict[i] = clf
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    

--- 141.25446939468384 seconds ---
--- 149.3139042854309 seconds ---
--- 155.10966753959656 seconds ---
--- 163.8846299648285 seconds ---
--- 170.8333821296692 seconds ---
--- 176.16339588165283 seconds ---
--- 182.76911807060242 seconds ---
--- 192.20374870300293 seconds ---
--- 189.81468439102173 seconds ---
--- 191.67213368415833 seconds ---


 ### New Query for Prediction

In [7]:
# Snowflake credentials stored in environment variables

username = os.getenv('Snowflake_User')
password = os.getenv('Snowflake_password')
account = os.getenv('Snowflake_account')

# Define warehouse, if neccessary
warehouse = 'DEVELOPER_BASIC'

# Define Database, if not defined in SQL request
#database = 'VESTA_STAGING'

# Create connection object for Snowflake connection
conn = sf.connect(user = username, password = password, account = account, warehouse = warehouse)

# Execution function
def execute_query(connection,query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close

try:
    # If defining a database, uncomment code set and add database in connection parameter
    #sql = 'use {}'.format(database)
    #execute_query(conn,sql)
    
    # Define warehouse to use in Snowflake
    sql = 'use warehouse {}'.format(warehouse)
    execute_query(conn,sql)
    
    print('Successful Connection')
    
    # Query to Snowflake
    sql = '''
WITH EDIP AS ( //This is sub table for a self join

    SELECT 
        *
    FROM "VESTA_DEVELOPMENT"."CLAIMS_REPORTING"."CCA_MEM_PROFILE_IP_ER_SNF" //THIS NEEDS TO CHANGE BASED ON CLIENT
    WHERE MEASURE = 'ED' or MEASURE = 'IP' 

    ),

EDIPTABLE AS ( //This table shows the Member ID, date start, and the number of ED/IP in the next 6 months

    SELECT
        SCORE.MEMBER_ID,
        TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) AS DATE_START,
        COUNT(DISTINCT EDIP.DOS_FROM) AS ED_IP_VISITS_IN_NEXT_6_MONTHS
    FROM "VESTA_DEVELOPMENT"."ANALYST_SANDBOX"."CLNT_STRAT_VIP" SCORE
        LEFT JOIN EDIP 
            ON SCORE.MEMBER_ID = EDIP.MEMBER_ID
                AND EDIP.DOS_FROM > TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01'))
                AND DATEDIFF(days,TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')),EDIP.DOS_FROM) >= 45
                AND DATEDIFF(days, TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')),EDIP.DOS_FROM) <= 180
    GROUP BY SCORE.MEMBER_ID,
        DATE_START 

    )

SELECT
    TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) AS DATE_START,
    //DATA_DATE_START,
    SCORE.MEMBER_ID,
    CASE WHEN EDIPTABLE.ED_IP_VISITS_IN_NEXT_6_MONTHS > 0 THEN 1 ELSE 0 END as ED_IP_VISIT,
    ACSC__COUNT,
    ACSC__SCORE,
    ACSC_A_FIB_AND_FLUTTER,
    ACSC_ALCOHOL_RELATED,
    ACSC_ANEMIA,
    ACSC_ANGINA,
    ACSC_ASTHMA,
    ACSC_CELLULITIS,
    ACSC_CONGESTIVE_HEART_FAILURE,
    ACSC_CONSTIPATION,
    ACSC_CONVULSION_EPILEPSY,
    ACSC_COPD,
    ACSC_DECUBITI_STAGE_3_,
    ACSC_DEHYDRATION_GASTROENTERITIS,
    ACSC_DIABETES_COMPLICATIONS,
    ACSC_DYSPEPSIA,
    ACSC_ENT_INFECTION,
    ACSC_HYPERTENSION,
    ACSC_HYPOGLYCEMIA,
    ACSC_HYPOKALEMIA,
    ACSC_INFLUENZA_PNEUMONIA,
    ACSC_MIGRAINE_HEADACHE,
    ACSC_NUTRITION_DEFICIENT,
    ACSC_PERFORATED_BLEEDING_ULCER,
    ACSC_PROXIMAL_FEMUR_FRACTURE,
    ACSC_PYELONEPHRITIS,
    ACSC_UTI,
    ACSC_VACCINE_PREVENTABLE_DX,
    DATEDIFF(year,DOB, TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01'))) as AGE,
    AMB_ACSC_COST,
    AMB_ACSC_COUNT,
    BH__COUNT,
    BH__SCORE,
    BH_ALTERED_MENTAL_STATE,
    BH_ALZHEIMERS_DEMENTIA,
    BH_ANXIETY,
    BH_BI_POLAR,
    BH_DEPRESSION,
    BH_SCHIZOPHRENIA,
    BH_SUBABUSE,
    CRN__COUNT,
    CRN_AFIB,
    CRN_ASTHMA,
    CRN_CARDIOVASCULAR_DX,
    CRN_CHRONIC_KIDNEY_DISEASE,
    CRN_CONGESTIVE_HEART_FAILURE,
    CRN_COPD,
    CRN_DIABETES_W__ACUTE_COMP,
    CRN_DIABETES_W__CHRONIC_COMP,
    CRN_DIABETES_W_OUT_COMP,
    CRN_FALLS,
    CRN_GASTRO_ESOPH_REFLUX,
    CRN_HIP_FRACTURE,
    CRN_HTN,
    CRN_OBESITY,
    CRN_OSTEOPOROSIS,
    CRN_PARKINSONS_DISEASE,
    CRN_PRESSURE_ULCER,
    CRN_PRIOR_MI,
    CRN_PRIOR_STROKE,
    CRN_SCORE,
    CRN_SLEEP_APNEA,
    CRN_SMOKING,
    CRN_UTI,
    DYAD_CKD_DD,
    DYAD_CKD_OP,
    DYAD_COPD_DD,
    DYAD_COPD_HF,
    DYAD_COPD_OP,
    DYAD_COUNT,
    DYAD_DM_CKD,
    DYAD_DM_OP,
    DYAD_HBP_HF,
    DYAD_HF_CKD,
    ED_ACSC_COST,
    ED_ACSC_COUNT,
    GENDER,
    CASE WHEN "GROUP" = 'E' THEN 'A' ELSE "GROUP" END as "GROUP",
    HMKR_ACSC_COST,
    HMKR_ACSC_COUNT,
    HTI_RISK_SCORE_V2_1,
    IP_ACSC_COST,
    IP_ACSC_COUNT,
    IP_READMIT_ACSC_COST,
    IP_READMIT_ACSC_COUNT,
    IP_RHB_ACSC_COST,
    IP_RHB_ACSC_COUNT,
    CASE WHEN LANGUAGE_SPOKEN is NULL THEN 'Unknown'
         WHEN LANGUAGE_SPOKEN = 'English' THEN 'English'
         WHEN LANGUAGE_SPOKEN = 'Chinese' THEN 'Chinese'
         WHEN LANGUAGE_SPOKEN = 'Spanish' THEN 'Spanish'
         WHEN LANGUAGE_SPOKEN = 'Russian' THEN 'Russian'
    ELSE 'Other' END AS LANGUAGE_SPOKEN_CLEAN,
    NI_COST_DENT,
    NI_COST_ED,
    NI_COST_HM,
    NI_COST_HMKR,
    NI_COST_HS,
    NI_COST_IP,
    NI_COST_IP_RHB,
    NI_COST_OP,
    NI_COST_OTH,
    NI_COST_PCA_T1020,
    NI_COST_PCA_T1019,
    NI_COST_PR,
    NI_COST_PSYC,
    NI_COST_RX,
    NI_COUNT_DENT,
    NI_COUNT_ED,
    NI_COUNT_HM,
    NI_COUNT_HMKR,
    NI_COUNT_HS,
    NI_COUNT_IP,
    NI_COUNT_IP_RHB,
    NI_COUNT_OP,
    NI_COUNT_OTH,
    NI_COUNT_PCA_T1020,
    NI_COUNT_PCA_T1019,
    NI_COUNT_PR,
    NI_COUNT_PSYC,
    NI_COUNT_RX,
    NON_IMPACTABLE_CLAIM_COUNT,
    OP_ACSC_COST,
    OP_ACSC_COUNT,
    CAST(PART_C_RISK_SCORE as FLOAT) as PART_C_RISK_SCORE,
    PCA_T1020_ACSC_COUNT,
    PCA_T1020_ACSC_COST,
    PCA_T1019_ACSC_COUNT,
    PCA_T1019_ACSC_COST,
    PR_ACSC_COST,
    PR_ACSC_COUNT,
    CASE WHEN RC is NULL THEN 'UNDEFINED' ELSE RC END AS RC_CLEAN,
    SNF_COST,
    SNF_COUNT,
    TOTAL_IMPACTABLE_COST,
    TOTAL_IMPACTABLE_COST_PRO,
    TOTAL_NON_IMPACTABLE_COST
FROM "VESTA_DEVELOPMENT"."ANALYST_SANDBOX"."CLNT_STRAT_VIP" SCORE
    LEFT JOIN EDIPTABLE
        ON SCORE.MEMBER_ID = EDIPTABLE.MEMBER_ID
            AND TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) = EDIPTABLE.DATE_START
WHERE SCORE.CLNT = 'CCA' //THIS NEEDS TO CHANGE BASED ON CLIENT
    AND TO_DATE(CONCAT(LEFT(SCORE.DATA_DATE_START,4),'-',RIGHT(SCORE.DATA_DATE_START,2),'-01')) 
        = (SELECT max(TO_DATE(CONCAT(LEFT(DATA_DATE_START,4),'-',RIGHT(DATA_DATE_START,2),'-01'))) FROM "VESTA_DEVELOPMENT"."ANALYST_SANDBOX"."CLNT_STRAT_VIP" WHERE CLNT = 'CCA') 
        // Most Recent Stratification for Client// THIS NEEDS TO CHANGE BASED ON CLIENT'''
    cursor = conn.cursor()
    cursor.execute(sql)
    
    # Dataframe creation
    test_df = pd.DataFrame.from_records(iter(cursor), columns = [x[0] for x in cursor.description])
    
    print('Successful DataFrame Created')
    
    cursor.close
    
except Exception as e:
        print(e)
        
finally:
    conn.close
    
print('Ready for Cleaning')  

print(test_df.shape)
test_df.head()


Successful Connection
Successful DataFrame Created
Ready for Cleaning
(2037, 135)


,DATE_START,MEMBER_ID,ED_IP_VISIT,ACSC__COUNT,ACSC__SCORE,ACSC_A_FIB_AND_FLUTTER,ACSC_ALCOHOL_RELATED,ACSC_ANEMIA,ACSC_ANGINA,ACSC_ASTHMA,...,PCA_T1019_ACSC_COUNT,PCA_T1019_ACSC_COST,PR_ACSC_COST,PR_ACSC_COUNT,RC_CLEAN,SNF_COST,SNF_COUNT,TOTAL_IMPACTABLE_COST,TOTAL_IMPACTABLE_COST_PRO,TOTAL_NON_IMPACTABLE_COST
0,2022-11-01,5365550560,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NHC,0.0,0.0,3049.21,3049.21,15034.33
1,2022-11-01,5366039898,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NHC,0.0,0.0,0.00,0.00,6322.19
2,2022-11-01,5365555823,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NHC,0.0,0.0,0.00,0.00,42688.35
3,2022-11-01,5365591632,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NHC,0.0,0.0,0.00,0.00,32280.25
4,2022-11-01,5365655530,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NHC,0.0,0.0,0.00,0.00,9505.96


### Cleaning Operations

In [8]:
# Remove Member ID and Data Start Date from Data Frame for prediction
data_date_start_list = test_df['DATE_START']
test_df = test_df.drop(['DATE_START'], axis = 1)
member_id_list = test_df['MEMBER_ID']
test_df = test_df.drop(['MEMBER_ID'], axis = 1)

#Remove ED_Visit from Data Frame
y_test_df = test_df['ED_IP_VISIT']
test_df = test_df.drop(['ED_IP_VISIT'], axis = 1)

# Find features with missing values
sumdf = pd.DataFrame(test_df.isna().sum())

features_to_drop = []
for row in sumdf.iterrows():
    if row[1][0] != 0:
        features_to_drop.append(row[0])

# Review features with missing values
print('These are the features that initially had missing values within the data frame: \n',features_to_drop,'\n\n')

Nulls_to_correct = ['ACSC_A_FIB_AND_FLUTTER', 
                    'ACSC_ALCOHOL_RELATED',
                    'ACSC_ANEMIA',
                    'ACSC_ANGINA',
                    'ACSC_ASTHMA',
                    'ACSC_CELLULITIS',
                    'ACSC_CONSTIPATION',
                    'ACSC_CONVULSION_EPILEPSY',
                    'ACSC_COPD',
                    'ACSC_DECUBITI_STAGE_3_',
                    'ACSC_DEHYDRATION_GASTROENTERITIS',
                    'ACSC_DYSPEPSIA',
                    'ACSC_ENT_INFECTION',
                    'ACSC_HYPERTENSION',
                    'ACSC_HYPOGLYCEMIA',
                    'ACSC_HYPOKALEMIA',
                    'ACSC_MIGRAINE_HEADACHE',
                    'ACSC_NUTRITION_DEFICIENT',
                    'ACSC_PERFORATED_BLEEDING_ULCER',
                    'ACSC_PROXIMAL_FEMUR_FRACTURE',
                    'ACSC_PYELONEPHRITIS',
                    'ACSC_VACCINE_PREVENTABLE_DX',
                    'CRN_FALLS',
                    'CRN_PRIOR_MI',
                    'CRN_SMOKING',
                    'NI_COST_HS',
                    'NI_COST_IP_RHB',
                    'NI_COST_OTH',
                    'NI_COST_PCA_T1020',
                    'NI_COST_PSYC',
                    'NI_COUNT_HS',
                    'NI_COUNT_IP_RHB',
                    'NI_COUNT_OTH',
                    'NI_COUNT_PCA_T1020',
                    'NI_COUNT_PSYC'] 

# Fill selected features with 0 value
for col in Nulls_to_correct:
    test_df[col] = test_df[col].fillna(0)
    
# Check for missing values and drop columns with missing values
sumdf = pd.DataFrame(test_df.isna().sum())

features_to_drop = []
for row in sumdf.iterrows():
    if row[1][0] != 0:
        features_to_drop.append(row[0])

print('These columns were dropped after because missing values were not corrected :\n', features_to_drop,'\n\n')        
test_df = test_df.drop(columns = features_to_drop)

# Convert object datatypes to dummy variables
object_list = []

for col in test_df.columns:
    if test_df[col].dtypes == 'object':
        object_list.append(col)
        
print('These are the features that were converted to dummy variables: \n',object_list,'\n\n')
test_df = pd.get_dummies(test_df, columns = object_list, drop_first = True)

test_df.head()

These are the features that initially had missing values within the data frame: 
 ['ACSC_CELLULITIS', 'ACSC_CONSTIPATION', 'ACSC_ENT_INFECTION', 'ACSC_HYPOGLYCEMIA', 'ACSC_VACCINE_PREVENTABLE_DX', 'NI_COST_PSYC', 'NI_COUNT_PSYC'] 


These columns were dropped after because missing values were not corrected :
 [] 


These are the features that were converted to dummy variables: 
 ['GENDER', 'GROUP', 'LANGUAGE_SPOKEN_CLEAN', 'RC_CLEAN'] 




,ACSC__COUNT,ACSC__SCORE,ACSC_A_FIB_AND_FLUTTER,ACSC_ALCOHOL_RELATED,ACSC_ANEMIA,ACSC_ANGINA,ACSC_ASTHMA,ACSC_CELLULITIS,ACSC_CONGESTIVE_HEART_FAILURE,ACSC_CONSTIPATION,...,GROUP_B,GROUP_C,GROUP_D,LANGUAGE_SPOKEN_CLEAN_Other,LANGUAGE_SPOKEN_CLEAN_Russian,LANGUAGE_SPOKEN_CLEAN_Spanish,LANGUAGE_SPOKEN_CLEAN_Unknown,RC_CLEAN_Institutional,RC_CLEAN_NHC,RC_CLEAN_Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,...,0,1,0,0,0,1,0,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,1,0


In [9]:
#Scale data for prediction with original model
X = test_df
X = scaler.fit_transform(X)

In [10]:
#Create Data Frame to store results
model_df = pd.DataFrame()

for i,clf in forest_dict.items():
    start_time = time.time()
    np.random.seed(i)
    model_df[i+1] = clf.predict_proba(X)[:,1]
    print("--- %s seconds ---" % (time.time() - start_time))

model_df  

--- 12.00824236869812 seconds ---
--- 15.29495906829834 seconds ---
--- 25.17923617362976 seconds ---
--- 20.79357361793518 seconds ---
--- 4.0376129150390625 seconds ---
--- 1.0676002502441406 seconds ---
--- 0.6540634632110596 seconds ---
--- 1.8370513916015625 seconds ---
--- 12.405155420303345 seconds ---
--- 0.7447271347045898 seconds ---


,1,2,3,4,5,6,7,8,9,10
0,0.255500,0.246000,0.241500,0.224500,0.259500,0.234500,0.265000,0.232500,0.262500,0.238000
1,0.291179,0.350667,0.333296,0.322452,0.319961,0.332547,0.337322,0.297000,0.325634,0.289396
2,0.364500,0.377500,0.423000,0.358000,0.401500,0.366500,0.408000,0.406000,0.415000,0.390500
3,0.293000,0.341985,0.315349,0.325500,0.324771,0.335500,0.294357,0.333759,0.315583,0.332383
4,0.311000,0.278000,0.317500,0.287000,0.290500,0.325500,0.321000,0.321000,0.319000,0.303500
...,...,...,...,...,...,...,...,...,...,...
2032,0.183000,0.158000,0.165000,0.170500,0.155000,0.151000,0.145417,0.196500,0.159500,0.197500
2033,0.150500,0.134500,0.168000,0.147500,0.166750,0.139000,0.137000,0.169500,0.126167,0.126000
2034,0.283500,0.323000,0.277750,0.300500,0.263000,0.278500,0.277667,0.229000,0.249500,0.250000
2035,0.363500,0.257500,0.320000,0.307500,0.288000,0.317000,0.256375,0.240000,0.318000,0.313000


In [11]:
model_df['VIP_SCORE'] = (model_df[1] + model_df[2] + model_df[3] + model_df[4] + 
    model_df[5] +model_df[6] + model_df[7] + model_df[8] + model_df[9] + model_df[10])/10

model_df

,1,2,3,4,5,6,7,8,9,10,VIP_SCORE
0,0.255500,0.246000,0.241500,0.224500,0.259500,0.234500,0.265000,0.232500,0.262500,0.238000,0.245950
1,0.291179,0.350667,0.333296,0.322452,0.319961,0.332547,0.337322,0.297000,0.325634,0.289396,0.319945
2,0.364500,0.377500,0.423000,0.358000,0.401500,0.366500,0.408000,0.406000,0.415000,0.390500,0.391050
3,0.293000,0.341985,0.315349,0.325500,0.324771,0.335500,0.294357,0.333759,0.315583,0.332383,0.321219
4,0.311000,0.278000,0.317500,0.287000,0.290500,0.325500,0.321000,0.321000,0.319000,0.303500,0.307400
...,...,...,...,...,...,...,...,...,...,...,...
2032,0.183000,0.158000,0.165000,0.170500,0.155000,0.151000,0.145417,0.196500,0.159500,0.197500,0.168142
2033,0.150500,0.134500,0.168000,0.147500,0.166750,0.139000,0.137000,0.169500,0.126167,0.126000,0.146492
2034,0.283500,0.323000,0.277750,0.300500,0.263000,0.278500,0.277667,0.229000,0.249500,0.250000,0.273242
2035,0.363500,0.257500,0.320000,0.307500,0.288000,0.317000,0.256375,0.240000,0.318000,0.313000,0.298088


In [12]:
#Merge Results back to Test Data Frame
test_df['VIP_SCORE'] = model_df['VIP_SCORE']
test_df['ED_IP_VISIT'] = y_test_df
test_df['MEMBER_ID'] = member_id_list
test_df['DATA_DATE_START'] = data_date_start_list

In [13]:
#View Predicted Members DF
predicted_df = test_df[['MEMBER_ID','VIP_SCORE','ED_IP_VISIT','DATA_DATE_START']]
predicted_df = predicted_df.sort_values(by=['VIP_SCORE'],ascending = False)
predicted_df.head(15)

,MEMBER_ID,VIP_SCORE,ED_IP_VISIT,DATA_DATE_START
620,5365587178,0.774619,0,2022-11-01
440,5365831518,0.754253,0,2022-11-01
859,5365784285,0.749605,0,2022-11-01
841,5364525796,0.749409,0,2022-11-01
265,5365907435,0.723603,0,2022-11-01
825,5365582644,0.715242,0,2022-11-01
610,5365585495,0.701632,0,2022-11-01
183,5365791675,0.675050,0,2022-11-01
346,5365619083,0.669218,0,2022-11-01
663,5365689546,0.659492,0,2022-11-01


In [14]:
#View Predicted Members DF
predicted_df = test_df[['MEMBER_ID','VIP_SCORE','ED_IP_VISIT','DATA_DATE_START']]
predicted_df = predicted_df.sort_values(by=['VIP_SCORE'],ascending = False)
predicted_df.head(15)

,MEMBER_ID,VIP_SCORE,ED_IP_VISIT,DATA_DATE_START
620,5365587178,0.774619,0,2022-11-01
440,5365831518,0.754253,0,2022-11-01
859,5365784285,0.749605,0,2022-11-01
841,5364525796,0.749409,0,2022-11-01
265,5365907435,0.723603,0,2022-11-01
825,5365582644,0.715242,0,2022-11-01
610,5365585495,0.701632,0,2022-11-01
183,5365791675,0.675050,0,2022-11-01
346,5365619083,0.669218,0,2022-11-01
663,5365689546,0.659492,0,2022-11-01


In [ ]:
# # convert from object to datetime and format-not needed in CTL so check sql to see what diffs are
# predicted_df['DATA_DATE_START'] = predicted_df['DATA_DATE_START'].astype('datetime64[ns]')
# predicted_df.info()
# predicted_df['DATA_DATE_START'] = predicted_df['DATA_DATE_START'].dt.strftime('%Y%m')
# predicted_df.head(15)


In [ ]:
predicted_df.to_csv('CCA Model - 202208.csv')

### Output and Update Tables in Snowflake for Analysis

In [15]:
# Snowflake credentials stored in environment variables

username = os.getenv('Snowflake_User')
password = os.getenv('Snowflake_password')
account = os.getenv('Snowflake_account')


#Define parameters if neccessary
warehouse = os.getenv('Snowflake_warehouse')
database = os.getenv('Snowflake_database')
schema = os.getenv('Snowflake_schema')

#Create connection object for Snowflake connection
conn = sf.connect(user = username, password = password, account = account, warehouse = warehouse)

#Execution function
def execute_query(connection,query):
    cursor = connection.cursor()
    cursor.execute(query)
    cursor.close

try:
    sql = 'USE DATABASE {}'.format(database)
    execute_query(conn,sql)
    
    sql = 'USE SCHEMA {}.{}'.format(database,schema)
    execute_query(conn,sql)
    
    #Define warehouse to use in Snowflake
    sql = 'use warehouse {}'.format(warehouse)
    execute_query(conn,sql)
    
    print('Successful Connection')
    
    #Query to Snowflake
    sql = "CREATE TABLE IF NOT EXISTS VIP_SCORING (CLNT string,MEMBER_ID string, VIP_SCORE float ,DATA_DATE_START string)"
    cursor = conn.cursor()
    cursor.execute(sql)
    
    entry_list = []
    for row in predicted_df.iterrows():
        client = 'CCA'
        member_id = str(row[1][0])
        vip_score = row[1][1]
        data_date = str(row[1][3])
        entry = (client,member_id,vip_score,data_date)
        entry_list.append(entry)       
    entry = str(entry_list)[1:len(str(entry_list))-1]
    sql = 'INSERT INTO VIP_SCORING (CLNT,MEMBER_ID, VIP_SCORE, DATA_DATE_START) VALUES {}'.format(entry)
    cursor = conn.cursor()
    cursor.execute(sql)  
    cursor.close

    print('Database updated')
    
except Exception as e:
        print(e)
        
finally:
    conn.close

Successful Connection
Database updated


# ITEMS BELOW THIS ARE FOR ANALYSIS ONLY

In [ ]:
#Prediction List
for row in predicted_df.iterrows():
    print('Member: ',row[1][0],' VIP_SCORE: ',round(row[1][1],4), ' ED_IP_VISIT: ',row[1][2])

In [ ]:
final_threshold = 0.3

final_y_pred = [1 if result >= final_threshold else 0 for result in test_df['VIP_SCORE'].tolist()]

# Creating confusion maxtrix
new_cnf_matrix = metrics.confusion_matrix(y_test_df,final_y_pred)

%matplotlib inline
class_names=[0,1]
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# Create heatmap
sns.heatmap(pd.DataFrame(new_cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")

# Axis labels
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

# Show the plot
plt.show()

#Calulating Metrics
accuracy = metrics.accuracy_score(y_test_df, final_y_pred)
precision = metrics.precision_score(y_test_df, final_y_pred)
recall = metrics.recall_score(y_test_df, final_y_pred)
    
print('Thereshold: ',final_threshold)
print("Accuracy: ",accuracy)
print("Precision: ",precision)
print("Recall: ",recall)

In [ ]:
#Realtime Accuracy
X = []
Y = []
count = 1
sums = 0
for row in predicted_df.iterrows():
    sums += row[1][2]
    accuracy = round(sums/count *100,2)
    X.append(count)
    Y.append(accuracy)
    print('After',count, 'prediction, the realtime accuarcy is ',accuracy)
    count +=1
    
plt.plot(X, Y, marker='.', label='Random Forest')

acc = np.where(np.array(Y) <= 60)

acc = np.where(np.array(Y) <= 60)
#for i,x in enumerate(acc[0]):
    #print(i,x)


### Important Features

In [ ]:
feature_list = [x for x in df.columns if x != 'ED_IP_VISIT']

importance_df = pd.DataFrame()

for i,clf in forest_dict.items():
    start_time = time.time()
    importances = list(clf.feature_importances_)
    feature_importances = [(feature, round(importance, 15)) for feature, importance in zip(feature_list, importances)]
    importance_df[i+1] = feature_importances
    print("--- %s seconds ---" % (time.time() - start_time))

importance_avg_list = []

for row in importance_df.iterrows():
    feature_name = row[1][1][0]
    avg = round((row[1][1][1] + row[1][2][1] + row[1][3][1] + row[1][4][1] + row[1][5][1] + row[1][6][1] + row[1][7][1] 
            + row[1][8][1] + row[1][9][1] + row[1][10][1])/10,4)
    
    importance_avg_list.append((feature_name,avg))
    
# Sort the feature importances by most important first
feature_importances = sorted(importance_avg_list, key = lambda x: x[1], reverse = True)

feature_importances[0:10]
    

### Random Search with Cross Validation - Hyperparameter Exploration

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier(random_state = 22822)

from sklearn.metrics import make_scorer

scoring = {"Accuracy":make_scorer(metrics.accuracy_score),"Precision":make_scorer(metrics.precision_score),
            "Recall":make_scorer(metrics.recall_score),"AUC":make_scorer(metrics.roc_auc_score)}
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 100, scoring='balanced_accuracy', 
                              cv = 3, verbose=6, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train,y_train);

#Best parameters found
rf_random.best_params_

In [ ]:
rf_random.cv_results_